In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 1 : data management: get started with BigQuery datasets

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/ml_ops_stage1/get_started_bq_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/ml_ops_stage1/get_started_bq_datasets.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 1 : data management: get started with BigQuery datasets.

### Dataset

The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset you use only the fields year, month and day to predict the value of mean daily temperature (mean_temp).

### Objective

In this tutorial, you learn how to use `BigQuery` as a dataset for training with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `Vertex Datasets`
- `BigQuery Datasets`

The steps performed include:

- Create a Vertex `Dataset` resource from `BigQuery` table -- compatible for `AutoML` training.
- Extract a copy of the dataset from `BigQuery` to a CSV file in Cloud Storage -- compatible for `AutoML` or custom training.
- Select rows from a `BigQuery` dataset into a `pandas` dataframe -- compatible for custom training.
- Select rows from a `BigQuery` dataset into a `tf.data.Dataset` -- compatible for custom training `TensorFlow` models.
- Select rows from extracted CSV files into a `tf.data.Dataset` -- compatible for custom training `TensorFlow` models.
- Create a BigQuery dataset from CSV files.

### Recommendations

When doing E2E MLOps on Google Cloud, the following best practices with structured (tabular) data in BigQuery:

- For AutoML training:
  - Create a managed dataset with Vertex `TabularDataset`.
  - Use the BigQuery table as the input to the dataset.
  - Specify columns and columns transformations when running the AutoML training pipeline job.


- For custom training:
  - For small datasets:
    - Extract the BigQuery to a pandas dataframe.
    - Preprocess the data in the dataframe.
  - For large datasets:
    - Create a tf.data.dataset generator from the BigQuery table.
    - Specify the columns for the custrom training.
    - Preprocess the data either:
      - Within the generator (upstream)
      - Within the model (downstream)


- Alternately:
    - Extract the BigQuery table to CSV files.
    - Preprocess the CSV files.
    - Create a tf.data.dataset generator from the CSV files.

## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os


# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *BigQuery* library as well.

In [ ]:
! pip3 install -U google-cloud-bigquery $USER_FLAG

Install the latest GA version of *TensorFlow IO* library as well.

In [ ]:
! pip3 install -U tensorflow-io pyarrow $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = 'us-central1'  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a custom training job using the Vertex SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. You can then
create an `Endpoint` resource based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

#### Import BigQuery

Import the BigQuery package into your Python environment.

In [ ]:
from google.cloud import bigquery

### Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, location=REGION)

### Create BigQuery client

Create the BigQuery client.

In [ ]:
bqclient = bigquery.Client()

#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = 'bigquery-public-data.samples.gsod'

### Create the Dataset

#### BigQuery input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `bq_source`: Import data items from a BigQuery table into the `Dataset` resource.

Learn more about [TabularDataset from BigQuery table](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python).

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="NOAA historical weather data" + "_" + TIMESTAMP,
    bq_source=[IMPORT_FILE]
)

label_column="mean_temp"

print(dataset.resource_name)

### Copy the dataset to Cloud Storage

Next, you make a copy of the BigQuery dataset, as a CSV file, to Cloud Storage using the BigQuery extract command.

Learn more about [BigQuery command line interface](https://cloud.google.com/bigquery/docs/reference/bq-cli-reference).

In [ ]:
! bq --location=us extract --destination_format CSV bigquery-public-data:samples.gsod $BUCKET_NAME/mydata*.csv

IMPORT_FILES = ! gsutil ls $BUCKET_NAME/mydata*.csv

print(IMPORT_FILES)

EXAMPLE_FILE = IMPORT_FILES[0]

! gsutil cat $EXAMPLE_FILE | head

### Create the Dataset

#### CSV input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.

Learn more about [TabularDataset from CSV files](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_gcs_sample-python)

In [ ]:
if 'IMPORT_FILES' in globals():
    gcs_source = IMPORT_FILES
else:
    gcs_source = [IMPORT_FILE]

dataset = aip.TabularDataset.create(
    display_name="NOAA historical weather data" + "_" + TIMESTAMP,
    gcs_source=gcs_source
)


label_column="mean_temp"

print(dataset.resource_name)

### Create a view of the BigQuery dataset

Alternatively, you can create a logical view of a BigQuery dataset that has a subset of the fields.

Learn more about [Creating BigQuery views](https://cloud.google.com/bigquery/docs/views).

In [ ]:
BQ_MY_DATASET = 'mydataset'
BQ_MY_TABLE = 'myview'
! bq --location=US mk -d \
$PROJECT_ID:$BQ_MY_DATASET

sql_script = f'''
CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_MY_DATASET}.{BQ_MY_TABLE}`
AS SELECT station_number,year,month,day,mean_temp FROM `{BQ_TABLE}`
'''
print(sql_script)

query = bqclient.query(sql_script)

### Read the BigQuery dataset into a pandas dataframe

Next, you read a sample of the dataset into a pandas dataframe using BigQuery `list_rows()` and `to_dataframe()` method, as follows:

- `list_rows()`: Performs a query on the specified table and returns a row iterator to the query results. Optionally specify:
 - `selected_fields`: Subset of fields (columns) to return.
 - `max_results`: The maximum number of rows to return. Same as SQL LIMIT command.


- `rows.to_dataframe()`: Invokes the row iterator and reads in the data into a pandas dataframe.

Learn more about [Loading BigQuery table into a dataframe](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.samples.gsod"
)

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ]

)

dataframe = rows.to_dataframe(
    # Optionally, explicitly request to use the BigQuery Storage API. As of
    # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
    # API is used by default.
    create_bqstorage_client=True,
)
print(dataframe.head())

### Read the BigQuery dataset into a tf.data.Dataset

Next, you read a sample of the dataset into a tf.data.Dataset using TensorFlow IO `BigQueryClient()` and `read_session()` method, with the following parameters:

- `parent`: Your project ID.
- `project_id`: The project ID of the BigQuery table.
- `dataset_id`: The ID of the BigQuery dataset.
- `table_id`. The ID of the table within the corresponding BigQuery dataset.
- `selected_fields`: Subset of fields (columns) to return.
- `output_types`: The output types of the corresponding fields.
- `requested_streams`: The number of parallel readers.

Learn more about [BigQuery TensorFlow reader](https://www.tensorflow.org/io/tutorials/bigquery).

Learn more about [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
from tensorflow.python.framework import dtypes

feature_names = "station_number,year,month,day".split(',')

target_name = "mean_temp"

def read_bigquery(project, dataset, table):
    tensorflow_io_bigquery_client = BigQueryClient()
    read_session = tensorflow_io_bigquery_client.read_session(
          parent="projects/" + PROJECT_ID,
          project_id=project,
          dataset_id=dataset,
          table_id=table,
          selected_fields=feature_names + [target_name],
          output_types=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
          requested_streams=2)

    dataset = read_session.parallel_read_rows()
    return dataset

PROJECT, DATASET, TABLE = IMPORT_FILE.split('/')[-1].split('.')
tf_dataset = read_bigquery(PROJECT, DATASET, TABLE)

print(tf_dataset.take(1))

### Read CSV files into a tf.data.Dataset

Alternatively, when your data is in CSV files, you can load the dataset into a tf.data.Dataset using `tf.data.experimental.CsvDataset`, with the following parameters:

- `filenames`: A list of one or more CSV files.
- `header`: Whether CSV file(s) contain a header.
- `select_cols`: Subset of fields (columns) to return.
- `record_defaults`: The output types of the corresponding fields.

Learn more about [tf.data CsvDataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/CsvDataset)

In [ ]:
import tensorflow as tf

feature_names = ["station_number,year,month,day".split(',')]

target_name = "mean_temp"

tf_dataset = tf.data.experimental.CsvDataset(
    filenames=IMPORT_FILES,
    header=True,
    select_cols=feature_names.append(target_name),
    record_defaults=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
)

print(tf_dataset.take(1))

### Create a BigQuery dataset from CSV files

You can create a BigQuery dataset from CSV files using the BigQuery `create_dataset()` and `load_table_from_uri()` methods, as follows:

- `create_dataset()`: Creates an empty BigQuery dataset, with the following parameters:
 - `dataset_ref`: The `DatasetReference` created from the dataset_id -- e.g., samples.
- `load_table_from_uri()`: Loads one or more CSV files into a table within the corresponding dataset, with the following parameters:
 - `url`: A set of one or more CVS files in Cloud Storage storage.
 - `table`: The `TableReference` for the table.
 - `job_config`: Specifications on how to load the CSV data.

Learn more about [Importing CSV data into BigQuery](https://www.tensorflow.org/io/tutorials/bigquery#import_census_data_into_bigquery).

In [ ]:
LOCATION = "us"

CSV_SCHEMA=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("wban_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
        bigquery.SchemaField("num_mean_temp_samples", "INTEGER"),
        bigquery.SchemaField("mean_dew_point", "FLOAT"),
        bigquery.SchemaField("num_mean_dew_point_samples", "INTEGER"),
        bigquery.SchemaField("mean_sealevel_pressure", "FLOAT"),
        bigquery.SchemaField("num_mean_sealevel_pressure_samples", "INTEGER"),
        bigquery.SchemaField("mean_station_pressure", "FLOAT"),
        bigquery.SchemaField("num_mean_station_pressure_samples", "INTEGER"),
        bigquery.SchemaField("mean_visibility", "FLOAT"),
        bigquery.SchemaField("num_mean_visibility_samples", "INTEGER"),
        bigquery.SchemaField("mean_wind_speed", "FLOAT"),
        bigquery.SchemaField("num_mean_wind_speed_samples", "INTEGER"),
        bigquery.SchemaField("max_sustained_wind_speed", "FLOAT"),
        bigquery.SchemaField("max_gust_wind_speed", "FLOAT"),
        bigquery.SchemaField("max_temperature", "FLOAT"),
        bigquery.SchemaField("max_temperature_explicit", "BOOLEAN"),
        bigquery.SchemaField("min_temperature", "FLOAT"),
        bigquery.SchemaField("min_temperature_explicit", "BOOLEAN"),
        bigquery.SchemaField("total_percipitation", "FLOAT"),
        bigquery.SchemaField("snow_depth", "FLOAT"),
        bigquery.SchemaField("fog", "BOOLEAN"),
        bigquery.SchemaField("rain", "BOOLEAN"),
        bigquery.SchemaField("snow", "BOOLEAN"),
        bigquery.SchemaField("hail", "BOOLEAN"),
        bigquery.SchemaField("thunder", "BOOLEAN"),
        bigquery.SchemaField("tornado", "BOOLEAN")
]


DATASET_ID = "samples"
TABLE_ID = "gsod"

def create_bigquery_dataset(dataset_id):
    dataset = bigquery.Dataset(bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id))
    dataset.location = "us"

    try:
        dataset = bqclient.create_dataset(dataset)  # API request
        return True
    except Exception as err:
        print(err)
        if err.code != 409: # http_client.CONFLICT
              raise
    return False

def load_data_into_bigquery(url, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.schema = CSV_SCHEMA
    job_config.skip_leading_rows = 1 # heading

    load_job = bqclient.load_table_from_uri(
      url, table, job_config=job_config
    )
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = bqclient.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))

load_data_into_bigquery(IMPORT_FILES, DATASET_ID, TABLE_ID)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if 'dataset' in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if 'model' in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if 'endpoint' in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if 'dag' in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if 'job' in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if 'batch_predict_job' in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if 'hpt_job' in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if 'BUCKET_NAME' in globals():
        ! gsutil rm -r $BUCKET_NAME